## SparkSession

In [3]:
import sys

sys.executable

'/opt/homebrew/opt/python@3.11/bin/python3.11'

In [4]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("instance").config().getOrCreate()
spark = SparkSession.builder.appName("instance").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/28 00:49:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Single column DataFrame from list

In [3]:
ages_list = [1,2,3,4,5]

from pyspark.sql.types import IntegerType, StringType
#spark.createDataFrame(ages_list)        #error
#spark.createDataFrame(ages_list, 'int') #ok

df = spark.createDataFrame(ages_list, IntegerType())
df.show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
+-----+



## spark.conf.set

In [4]:
#spark.conf.set('spark.sql.shuffle.partitions', '2')
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

value
1
2
3
4
5


In [5]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', False)
df

DataFrame[value: int]

## Help (documentation)

In [6]:
#help(spark.createDataFrame)
#spark.createDataFrame?
spark.read.csv?

Signature:
spark.read.csv(
    path: Union[str, List[str]],
    schema: Union[pyspark.sql.types.StructType, str, NoneType] = None,
    sep: Optional[str] = None,
    encoding: Optional[str] = None,
    quote: Optional[str] = None,
    escape: Optional[str] = None,
    comment: Optional[str] = None,
    header: Union[bool, str, NoneType] = None,
    inferSchema: Union[bool, str, NoneType] = None,
    ignoreLeadingWhiteSpace: Union[bool, str, NoneType] = None,
    ignoreTrailingWhiteSpace: Union[bool, str, NoneType] = None,
    nullValue: Optional[str] = None,
    nanValue: Optional[str] = None,
    positiveInf: Optional[str] = None,
    negativeInf: Optional[str] = None,
    dateFormat: Optional[str] = None,
    timestampFormat: Optional[str] = None,
    maxColumns: Union[str, int, NoneType] = None,
    maxCharsPerColumn: Union[str, int, NoneType] = None,
    maxMalformedLogPerPartition: Union[str, int, NoneType] = None,
    mode: Optional[str] = None,
    columnNameOfCorruptRecord: Opt

## Multi column DataFrame from list of tuples

In [7]:
user_list = [(1, 'Scott'), (2, 'Donald'), (3, 'Mickey'), (4, 'Elvis')]

In [8]:
df = spark.createDataFrame(user_list, 'user_id int, user_first_name string')
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- user_first_name: string (nullable = true)



## Row type

In [9]:
df.collect()

[Row(user_id=1, user_first_name='Scott'),
 Row(user_id=2, user_first_name='Donald'),
 Row(user_id=3, user_first_name='Mickey'),
 Row(user_id=4, user_first_name='Elvis')]

In [10]:
type(df.collect())    #list
type(df.collect()[1]) #pyspark.sql.types.Row

pyspark.sql.types.Row

In [11]:
from pyspark.sql import Row
#Row?

In [12]:
r = Row(name='Alice', age=11)
r1 =  Row('Alice', 11)
r, r1, r.name, r['name'], r[0]

(Row(name='Alice', age=11), <Row('Alice', 11)>, 'Alice', 'Alice', 'Alice')

## Multi column DataFrame from list of lists

In [13]:
user_list = [[1, 'Scott'], [2, 'Donald'], [3, 'Mickey'], [4, 'Elvis']]
df = spark.createDataFrame(user_list, 'user_id int, user_first_name string')
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- user_first_name: string (nullable = true)



## Multi column DataFrame by converting list of lists to list of rows (list comprehension)

In [14]:
from pyspark.sql import Row
user_list = [[1, 'Scott'], [2, 'Donald'], [3, 'Mickey'], [4, 'Elvis']]
user_rows = [Row(*user) for user in user_list]
spark.createDataFrame(user_rows, 'user_id int, user_first_name string')

DataFrame[user_id: int, user_first_name: string]

## Multi column DataFrame by converting list of tuples to list of rows (list comprehension)

In [19]:
from pyspark.sql import Row
user_tuples = [(1, 'Scott'), (2, 'Donald'), (3, 'Mickey'), (4, 'Elvis')]
user_rows = [Row(*user) for user in user_tuples]
spark.createDataFrame(user_rows, 'user_id int, user_first_name string')

DataFrame[user_id: int, user_first_name: string]

In [20]:
from pyspark.sql import Row
user_tuples = [(1, 'Scott'), (2, 'Donald'), (3, 'Mickey'), (4, 'Elvis')]
user_rows = [Row(*user) for user in user_tuples]
spark.createDataFrame(data=user_rows, schema='user_id int, user_first_name string')

DataFrame[user_id: int, user_first_name: string]

## star *

In [21]:
def dummy(*args):
    print(args)
    print(len(args))
    
t = (1,'Chris') 
l = [1,'Chris']
dummy(t), dummy(*t), dummy(l), dummy(*l)

((1, 'Chris'),)
1
(1, 'Chris')
2
([1, 'Chris'],)
1
(1, 'Chris')
2


(None, None, None, None)

## Multi column DataFrame from list of dicts

In [27]:
user_dict_list = [{'user_id': 1, 'user_name': 'Scott'}, 
                  {'user_id': 2, 'user_name': 'Donald'},
                  {'user_id': 3, 'user_name': 'Mickey'}, 
                  {'user_id': 4, 'user_name': 'Elvis'}]

spark.createDataFrame(user_dict_list) #deprecated

DataFrame[user_id: bigint, user_name: string]

In [32]:
from pyspark.sql import Row
user_details=user_dict_list[1]
type(user_details), user_details.values(), user_details.keys()


(dict, dict_values([2, 'Donald']), dict_keys(['user_id', 'user_name']))

In [38]:
user_rows=[Row(*user.values()) for user in user_dict_list]
spark.createDataFrame(user_rows, 'user_id bigint, user_first_name string')

DataFrame[user_id: bigint, user_first_name: string]

In [37]:
user_rows=[Row(**user) for user in user_dict_list]
spark.createDataFrame(user_rows)

DataFrame[user_id: bigint, user_name: string]

In [48]:
def dummy(**kwargs):
    print(kwargs)
    print(len(kwargs))

user_details={'user_id': 1, 'user_name': 'Scott'}

dummy(user_details=user_details), dummy(user_id=1, user_name='Scott'), dummy(**user_details)

{'user_details': {'user_id': 1, 'user_name': 'Scott'}}
1
{'user_id': 1, 'user_name': 'Scott'}
2
{'user_id': 1, 'user_name': 'Scott'}
2


(None, None, None)

## Basic data types

In [62]:
import datetime

users = [
    {
        'id': 1,
        'first_name': 'Corrie',
        'last_name': 'Van den Oord',
        'email': 'cvandenoord@etsy.com',
        'id_customer': True,
        'amount_paid': 1000.55,
        'customer_from': datetime.date(2021, 1, 15),
        'last_updated_ts': datetime.datetime(2021, 2, 18, 1, 15, 0)
    },
    {
        'id': 2,
        'first_name': 'Nicolaus',
        'last_name': 'Brewitt',
        'email': 'nbrewitt@dailymail.co.uk',
        'id_customer': None,
        'amount_paid': 900.0,
        'customer_from': datetime.date(2021, 2, 14),
        'last_updated_ts': datetime.datetime(2021, 2, 18, 3, 33, 0)
    }
]

In [63]:
from pyspark.sql import Row
users_df = spark.createDataFrame([Row(**user) for user in users])
users_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- id_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)



In [64]:
users_df.show() # None->null, True->true

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|id_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+



In [60]:
users_df.columns

['id',
 'first_name',
 'last_name',
 'email',
 'id_customer',
 'amount_paid',
 'customer_from',
 'last_updated_ts']

In [61]:
users_df.dtypes 

[('id', 'bigint'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('email', 'string'),
 ('id_customer', 'boolean'),
 ('amount_paid', 'double'),
 ('customer_from', 'date'),
 ('last_updated_ts', 'timestamp')]

## Specifying schema for DataFrame

In [85]:
import datetime

users = [(1, 'Corrie', 'Van den Oord', 'cvandenoord@etsy.com', True, 1000.55, datetime.date(2021, 1, 15), datetime.datetime(2021, 2, 18, 1, 15, 0)),
         (2, 'Nicolaus', 'Brewitt', 'nbrewitt@dailymail.co.uk', None, 900.0,  datetime.date(2021, 2, 14), datetime.datetime(2021, 2, 18, 3, 33, 0))]

users_schema = '''
    id bigint, 
    first_name string, 
    last_name string, 
    email string, 
    is_customer boolean, 
    amount_paid double, 
    customer_from date, 
    last_updated_ts timestamp
'''

users_df = spark.createDataFrame(users, users_schema)
users_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- is_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)



In [86]:
#Infer schema if the schema contains column names only
users_schema = [
    'id', 
    'first_name', 
    'last_name', 
    'email', 
    'is_customer', 
    'amount_paid', 
    'customer_from', 
    'last_updated_ts'
]

users_df = spark.createDataFrame(users, users_schema)
users_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- is_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)



In [87]:
from pyspark.sql.types import *

fields = StructType ([
    StructField('id', IntegerType()),
    StructField('first_name', StringType()),
    StructField('last_name', StringType()),
    StructField('email', StringType()),
    StructField('is_customer', BooleanType()),
    StructField('amount_paid', DoubleType()),
    StructField('customer_from', DateType()),
    StructField('last_udpated_ts', TimestampType()),    
])

users_df = spark.createDataFrame(users, fields)
users_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- is_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_udpated_ts: timestamp (nullable = true)



## Create DataFrame from pandas DataFrame

In [93]:
import pandas as pd # not specified values filled as nulls

users = [(1, 'Corrie', 'Van den Oord', 'cvandenoord@etsy.com', True, 1000.55, datetime.date(2021, 1, 15), datetime.datetime(2021, 2, 18, 1, 15, 0)),
         (2, 'Nicolaus', 'Brewitt', 'nbrewitt@dailymail.co.uk', None)]

pd.DataFrame(users)

,0,1,2,3,4,5,6,7
0,1,Corrie,Van den Oord,cvandenoord@etsy.com,True,1000.55,2021-01-15,2021-02-18 01:15:00
1,2,Nicolaus,Brewitt,nbrewitt@dailymail.co.uk,None,NaN,None,NaT


In [94]:
spark.createDataFrame(pd.DataFrame(users))

DataFrame[0: bigint, 1: string, 2: string, 3: string, 4: boolean, 5: double, 6: date, 7: timestamp]

## Spark special data types - ARRAY, STRUCT, MAP
* list and dict can be implicitly converted to ARRAY and MAP respectively
* spark API to be used to convert Python data types to STRUCT

## ARRAY (list -> array)

In [26]:
import datetime
from pyspark.sql import Row

users = [
    {
        'id': 1,
        'first_name': 'Corrie',
        'last_name': 'Van den Oord',
        'email': 'cvandenoord@etsy.com',
        'phone_numbers': ['+1 234 567 8901', '+1 234 567 8911'],
        'is_customer': True,
        'amount_paid': 1000.55,
        'customer_from': datetime.date(2021, 1, 15),
        'last_updated_ts': datetime.datetime(2021, 2, 18, 1, 15, 0)
    },
    {
        'id': 2,
        'first_name': 'Nicolaus',
        'last_name': 'Brewitt',
        'email': 'nbrewitt@dailymail.co.uk',
        'phone_numbers': ['+1 234 567 8923', '+1 234 567 8934'],
        'is_customer': None,
        'amount_paid': 900.0,
        'customer_from': datetime.date(2021, 2, 14),
        'last_updated_ts': datetime.datetime(2021, 2, 18, 3, 33, 0)
    },
    {
        'id': 3,
        'first_name': 'Kurt',
        'last_name': 'Rome',
        'email': 'krome4@shutterfly.co.uk',
        'phone_numbers': None,
        'is_customer': False,
        'amount_paid': None,
        'customer_from': datetime.date(2021, 2, 14),
        'last_updated_ts': datetime.datetime(2024, 2, 28, 5, 27, 0)
    }
]

In [27]:
users_df = spark.createDataFrame([Row(**user) for user in users])
users_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_numbers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)



In [28]:
users_df.show(truncate=False)
users_df.columns, users_df.dtypes

+---+----------+------------+------------------------+----------------------------------+-----------+-----------+-------------+-------------------+
|id |first_name|last_name   |email                   |phone_numbers                     |is_customer|amount_paid|customer_from|last_updated_ts    |
+---+----------+------------+------------------------+----------------------------------+-----------+-----------+-------------+-------------------+
|1  |Corrie    |Van den Oord|cvandenoord@etsy.com    |[+1 234 567 8901, +1 234 567 8911]|true       |1000.55    |2021-01-15   |2021-02-18 01:15:00|
|2  |Nicolaus  |Brewitt     |nbrewitt@dailymail.co.uk|[+1 234 567 8923, +1 234 567 8934]|NULL       |900.0      |2021-02-14   |2021-02-18 03:33:00|
|3  |Kurt      |Rome        |krome4@shutterfly.co.uk |NULL                              |false      |NULL       |2021-02-14   |2024-02-28 05:27:00|
+---+----------+------------+------------------------+----------------------------------+-----------+-----------

(['id',
  'first_name',
  'last_name',
  'email',
  'phone_numbers',
  'is_customer',
  'amount_paid',
  'customer_from',
  'last_updated_ts'],
 [('id', 'bigint'),
  ('first_name', 'string'),
  ('last_name', 'string'),
  ('email', 'string'),
  ('phone_numbers', 'array<string>'),
  ('is_customer', 'boolean'),
  ('amount_paid', 'double'),
  ('customer_from', 'date'),
  ('last_updated_ts', 'timestamp')])

In [29]:
from pyspark.sql.functions import explode
users_df.withColumn('phone_number', explode('phone_numbers')).drop('phone_numbers').show()

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+---------------+
| id|first_name|   last_name|               email|is_customer|amount_paid|customer_from|    last_updated_ts|   phone_number|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+---------------+
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|+1 234 567 8901|
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|+1 234 567 8911|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|+1 234 567 8923|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|+1 234 567 8934|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+---------------+


In [34]:
from pyspark.sql.functions import col
users_df.select('id', col('phone_numbers')[0].alias('mobile'), col('phone_numbers')[1].alias('home')).show()

+---+---------------+---------------+
| id|         mobile|           home|
+---+---------------+---------------+
|  1|+1 234 567 8901|+1 234 567 8911|
|  2|+1 234 567 8923|+1 234 567 8934|
|  3|           NULL|           NULL|
+---+---------------+---------------+



In [36]:
from pyspark.sql.functions import explode_outer
users_df.withColumn('phone_number', explode_outer('phone_numbers')).drop('phone_numbers').show()

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+---------------+
| id|first_name|   last_name|               email|is_customer|amount_paid|customer_from|    last_updated_ts|   phone_number|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+---------------+
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|+1 234 567 8901|
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|+1 234 567 8911|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|+1 234 567 8923|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|+1 234 567 8934|
|  3|      Kurt|        Rome|krome4@shutterfly...|      false|       NULL|   2021-02-14|2024-02-28 05:27:00|           NULL|


## MAP (dict -> map)

In [8]:
import datetime
users = [
    {
        'id': 1,
        'first_name': 'Corrie',
        'last_name': 'Van den Oord',
        'email': 'cvandenoord@etsy.com',
        'phone_numbers': {'mobile': '+1 234 567 8901', 'home': '+1 234 567 8911'},
        'is_customer': True,
        'amount_paid': 1000.55,
        'customer_from': datetime.date(2021, 1, 15),
        'last_updated_ts': datetime.datetime(2021, 2, 18, 1, 15, 0)
    },
    {
        'id': 2,
        'first_name': 'Nicolaus',
        'last_name': 'Brewitt',
        'email': 'nbrewitt@dailymail.co.uk',
        'phone_numbers': {'mobile': '+1 234 567 8923', 'home': '+1 234 567 8934'},
        'is_customer': None,
        'amount_paid': 900.0,
        'customer_from': datetime.date(2021, 2, 14),
        'last_updated_ts': datetime.datetime(2021, 2, 18, 3, 33, 0)
    },
    {
        'id': 3,
        'first_name': 'Kurt',
        'last_name': 'Rome',
        'email': 'krome4@shutterfly.co.uk',
        'phone_numbers': None,
        'is_customer': False,
        'amount_paid': None,
        'customer_from': datetime.date(2021, 2, 14),
        'last_updated_ts': datetime.datetime(2024, 2, 28, 5, 27, 0)
    }
]

In [13]:
from pyspark.sql import Row
users_df=spark.createDataFrame([Row(**user) for user in users])
users_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_numbers: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- is_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)



In [17]:
users_df.show()

+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|       phone_numbers|is_customer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|{mobile -> +1 234...|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|{mobile -> +1 234...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  3|      Kurt|        Rome|krome4@shutterfly...|                NULL|      false|       NULL|   2021-02-14|2024-02-28 05:27:00|
+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+



In [19]:
users_df.select('id', 'phone_numbers').show(truncate=False)

+---+----------------------------------------------------+
|id |phone_numbers                                       |
+---+----------------------------------------------------+
|1  |{mobile -> +1 234 567 8901, home -> +1 234 567 8911}|
|2  |{mobile -> +1 234 567 8923, home -> +1 234 567 8934}|
|3  |NULL                                                |
+---+----------------------------------------------------+



In [22]:
users_df.columns, users_df.dtypes

(['id',
  'first_name',
  'last_name',
  'email',
  'phone_numbers',
  'is_customer',
  'amount_paid',
  'customer_from',
  'last_updated_ts'],
 [('id', 'bigint'),
  ('first_name', 'string'),
  ('last_name', 'string'),
  ('email', 'string'),
  ('phone_numbers', 'map<string,string>'),
  ('is_customer', 'boolean'),
  ('amount_paid', 'double'),
  ('customer_from', 'date'),
  ('last_updated_ts', 'timestamp')])

In [35]:
from pyspark.sql.functions import col
users_df.select('id', col('phone_numbers')['mobile'].alias('mobile'), col('phone_numbers')['home'].alias('home')).show()

+---+---------------+---------------+
| id|         mobile|           home|
+---+---------------+---------------+
|  1|+1 234 567 8901|+1 234 567 8911|
|  2|+1 234 567 8923|+1 234 567 8934|
|  3|           NULL|           NULL|
+---+---------------+---------------+



In [37]:
from pyspark.sql.functions import explode
users_df.select('*', explode('phone_numbers')).withColumnRenamed('key', 'phone_type').withColumnRenamed('value', 'phone_number').drop('phone_numbers').show()

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+----------+---------------+
| id|first_name|   last_name|               email|is_customer|amount_paid|customer_from|    last_updated_ts|phone_type|   phone_number|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+----------+---------------+
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|    mobile|+1 234 567 8901|
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|      home|+1 234 567 8911|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|    mobile|+1 234 567 8923|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|      home|+1 234 567 8934|
+---+----------+------------+-------------------

In [39]:
from pyspark.sql.functions import explode_outer
users_df.select('*', explode_outer('phone_numbers')).withColumnRenamed('key', 'phone_type').withColumnRenamed('value', 'phone_number').drop('phone_numbers').show()

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+----------+---------------+
| id|first_name|   last_name|               email|is_customer|amount_paid|customer_from|    last_updated_ts|phone_type|   phone_number|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+----------+---------------+
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|    mobile|+1 234 567 8901|
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|       true|    1000.55|   2021-01-15|2021-02-18 01:15:00|      home|+1 234 567 8911|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|    mobile|+1 234 567 8923|
|  2|  Nicolaus|     Brewitt|nbrewitt@dailymai...|       NULL|      900.0|   2021-02-14|2021-02-18 03:33:00|      home|+1 234 567 8934|
|  3|      Kurt|        Rome|krome4@shutterfly..

## STRUCT - predefined structure using Row

In [5]:
import datetime
from pyspark.sql import Row

users = [
    {
        'id': 1,
        'first_name': 'Corrie',
        'last_name': 'Van den Oord',
        'email': 'cvandenoord@etsy.com',
        'phone_numbers': Row(mobile='+1 234 567 8901', home='+1 234 567 8911'),
        'is_customer': True,
        'amount_paid': 1000.55,
        'customer_from': datetime.date(2021, 1, 15),
        'last_updated_ts': datetime.datetime(2021, 2, 18, 1, 15, 0)
    },
    {
        'id': 2,
        'first_name': 'Nicolaus',
        'last_name': 'Brewitt',
        'email': 'nbrewitt@dailymail.co.uk',
        'phone_numbers': Row(mobile='+1 234 567 8923', home='+1 234 567 8934'),
        'is_customer': None,
        'amount_paid': 900.0,
        'customer_from': datetime.date(2021, 2, 14),
        'last_updated_ts': datetime.datetime(2021, 2, 18, 3, 33, 0)
    },
    {
        'id': 3,
        'first_name': 'Kurt',
        'last_name': 'Rome',
        'email': 'krome4@shutterfly.co.uk',
        'phone_numbers': Row(mobile=None, home=None),
        'is_customer': False,
        'amount_paid': None,
        'customer_from': datetime.date(2021, 2, 14),
        'last_updated_ts': datetime.datetime(2024, 2, 28, 5, 27, 0)
    }
]

In [6]:
users_df=spark.createDataFrame([Row(**user) for user in users])
users_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone_numbers: struct (nullable = true)
 |    |-- mobile: string (nullable = true)
 |    |-- home: string (nullable = true)
 |-- is_customer: boolean (nullable = true)
 |-- amount_paid: double (nullable = true)
 |-- customer_from: date (nullable = true)
 |-- last_updated_ts: timestamp (nullable = true)



24/05/28 00:49:22 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
users_df.show(truncate=False)

+---+----------+------------+------------------------+----------------------------------+-----------+-----------+-------------+-------------------+
|id |first_name|last_name   |email                   |phone_numbers                     |is_customer|amount_paid|customer_from|last_updated_ts    |
+---+----------+------------+------------------------+----------------------------------+-----------+-----------+-------------+-------------------+
|1  |Corrie    |Van den Oord|cvandenoord@etsy.com    |{+1 234 567 8901, +1 234 567 8911}|true       |1000.55    |2021-01-15   |2021-02-18 01:15:00|
|2  |Nicolaus  |Brewitt     |nbrewitt@dailymail.co.uk|{+1 234 567 8923, +1 234 567 8934}|NULL       |900.0      |2021-02-14   |2021-02-18 03:33:00|
|3  |Kurt      |Rome        |krome4@shutterfly.co.uk |{NULL, NULL}                      |false      |NULL       |2021-02-14   |2024-02-28 05:27:00|
+---+----------+------------+------------------------+----------------------------------+-----------+-----------

In [11]:
users_df.select('id', 'phone_numbers').show(truncate=False)

+---+----------------------------------+
|id |phone_numbers                     |
+---+----------------------------------+
|1  |{+1 234 567 8901, +1 234 567 8911}|
|2  |{+1 234 567 8923, +1 234 567 8934}|
|3  |{NULL, NULL}                      |
+---+----------------------------------+



In [13]:
users_df.columns, users_df.dtypes

(['id',
  'first_name',
  'last_name',
  'email',
  'phone_numbers',
  'is_customer',
  'amount_paid',
  'customer_from',
  'last_updated_ts'],
 [('id', 'bigint'),
  ('first_name', 'string'),
  ('last_name', 'string'),
  ('email', 'string'),
  ('phone_numbers', 'struct<mobile:string,home:string>'),
  ('is_customer', 'boolean'),
  ('amount_paid', 'double'),
  ('customer_from', 'date'),
  ('last_updated_ts', 'timestamp')])

In [15]:
users_df.select('id', 'phone_numbers.mobile', 'phone_numbers.home').show()

+---+---------------+---------------+
| id|         mobile|           home|
+---+---------------+---------------+
|  1|+1 234 567 8901|+1 234 567 8911|
|  2|+1 234 567 8923|+1 234 567 8934|
|  3|           NULL|           NULL|
+---+---------------+---------------+



In [19]:
from pyspark.sql.functions import col
users_df.select('id', col('phone_numbers')['mobile'], col('phone_numbers')['home']).show()

+---+--------------------+------------------+
| id|phone_numbers.mobile|phone_numbers.home|
+---+--------------------+------------------+
|  1|     +1 234 567 8901|   +1 234 567 8911|
|  2|     +1 234 567 8923|   +1 234 567 8934|
|  3|                NULL|              NULL|
+---+--------------------+------------------+



In [20]:
users_df.select('id', 'phone_numbers.*').show()

+---+---------------+---------------+
| id|         mobile|           home|
+---+---------------+---------------+
|  1|+1 234 567 8901|+1 234 567 8911|
|  2|+1 234 567 8923|+1 234 567 8934|
|  3|           NULL|           NULL|
+---+---------------+---------------+

